In [45]:
from ultralytics import YOLO
import numpy as np
import cv2
from collections import defaultdict

In [46]:
model = YOLO("yolov8x.pt")

video = cv2.VideoCapture("test_video.mp4")
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Цветовые модели HSV
hsv_orange_lower = np.array([10, 100, 100], dtype="uint8")
hsv_orange_upper = np.array([25, 255, 255], dtype="uint8")
hsv_green_lower = np.array([40, 50, 50], dtype="uint8")
hsv_green_upper = np.array([80, 255, 255], dtype="uint8")

# Создание видеозаписи
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('output.mp4', fourcc, 20.0, (width, height))

# Хранение траекторий в виде {1: [(10;20), (11;20), ...], ...}
track_history = defaultdict(lambda: [])
   

In [47]:
def person_has_helmet(det, frame):
    x1, y1, x2, y2 = det
    person_roi = frame[y1:int(y1 + (y2 - y1) / 3), x1:x2]  
    hsv = cv2.cvtColor(person_roi, cv2.COLOR_BGR2HSV)  
    mask_orange = cv2.inRange(hsv, hsv_orange_lower, hsv_orange_upper)
    mask_green = cv2.inRange(hsv, hsv_green_lower, hsv_green_upper)     
    return np.sum(mask_orange) > 100 or np.sum(mask_green) > 100

In [48]:
while(video.isOpened()):
    ret, frame = video.read()   
    if not ret:
        break

    results = model.track(frame, persist=True, conf=0.2, classes=[0])
        
    detections = results[0].boxes.xyxy.cpu().numpy().astype(int)
    track_ids = results[0].boxes.id.int().cpu().tolist()

    # наличие/отсутствие шлема у каждого обнаруженного объекта
    no_helmet = [not person_has_helmet(det, frame) for det in detections]
        
    # индексы людей без шлема    
    no_helmet_track_ids = [i for i, not_has_helmet in enumerate(no_helmet) if not_has_helmet]

    # рисование прямоугольников с индексами
    for index, det in enumerate(detections):
        x1, y1, x2, y2 = det
        if index in no_helmet_track_ids:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"ID: {index + 1}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {index + 1}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # рисование ломаных 
    for index in no_helmet_track_ids:
        x1, y1, x2, y2 = detections[index]
        x = (x1 + x2) // 2
        y = (y1 + y2) // 2
        track = track_history[index]
        track.append((x, y))
        if len(track) > 100: # храниться до 100 последних значений 
            track.pop(0)
        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(frame, [points], isClosed=False, color=(0, 0, 255), thickness=4)

    cv2.putText(frame, f'People: {len(detections)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Video", frame)
    writer.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
writer.release()
cv2.destroyAllWindows()



0: 384x640 4 persons, 2359.8ms
Speed: 3.0ms preprocess, 2359.8ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2324.8ms
Speed: 8.0ms preprocess, 2324.8ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2298.8ms
Speed: 2.0ms preprocess, 2298.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2285.8ms
Speed: 3.0ms preprocess, 2285.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2274.9ms
Speed: 3.0ms preprocess, 2274.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2261.0ms
Speed: 4.0ms preprocess, 2261.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2272.9ms
Speed: 3.0ms preprocess, 2272.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2477.9ms
Speed: 4.0ms preprocess, 2477.9ms inference, 3.0ms 